In [46]:
"from data import load_data_file, create_vocab
"
train_data = load_data_file("../data/GenBench/story")
eval_data = load_data_file("../data/GenBench/nonstory")

print(f"Loaded {len(train_data)} train lines and {len(eval_data)} eval lines")

Loaded 9487 train lines and 1946 eval lines


In [47]:
from tokenizer import WordLevelTokenizer

train_vocab = create_vocab([line.morphemes() for line in train_data], threshold=1)
tokenizer = WordLevelTokenizer(vocab=train_vocab, model_max_length=64)

In [48]:
eval_vocab = set([morpheme for line in eval_data for morpheme in line.morphemes() if morpheme != "[SEP]"])
unknown_eval_types = len([morpheme for morpheme in eval_vocab if morpheme not in train_vocab])
print(f"{unknown_eval_types} unknown token types in eval set out of {len(eval_vocab)} token types in eval")

eval_tokens = [morpheme for line in eval_data for morpheme in line.morphemes() if morpheme != "[SEP]"]
unknown_tokens = len([token for token in eval_tokens if token not in train_vocab])
print(
    f"{unknown_tokens} unknown tokens in eval data out of {len(eval_tokens)} total tokens ({unknown_tokens / len(eval_tokens)})")

339 unknown token types in eval set out of 1324 token types in eval
597 unknown tokens in eval data out of 12363 total tokens (0.04828925018199466)


In [22]:
from data import prepare_dataset, create_gloss_vocab
from uspanteko_morphology import morphology as morphology_tree

glosses = create_gloss_vocab(morphology_tree)
train = prepare_dataset(data=train_data, tokenizer=tokenizer, labels=glosses, device="cpu")
eval = prepare_dataset(data=train_data, tokenizer=tokenizer, labels=glosses, device="cpu")

  0%|          | 0/5843 [00:00<?, ?ex/s]

  0%|          | 0/5843 [00:00<?, ?ex/s]

In [43]:
import random

all_data = train_data + eval_data
random_split_train = random.sample(all_data, len(eval_data))
random_split_eval = [line for line in all_data if line not in random_split_train]
print(f"Split randomly into {len(random_split_train)} train points and {len(random_split_eval)} eval points")

Split randomly into 5590 train points and 5843 eval points


5843